# Data testing

##### Imports and initialization

In [4]:
%matplotlib inline

import os,sys
#path = os.path.realpath(os.path.join(os.getcwd(), ".."))
path = "/home/zantyr/Denoising"
sys.path.append(path)
os.chdir(path)

import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy.io.wavfile as sio
import tempfile

from IPython.display import Image, Audio, display

import keras
from keras.utils.vis_utils import plot_model
from keras.layers         import Dropout, Input, Conv1D, Conv2D, AveragePooling2D, Flatten, Dense, Deconv2D, UpSampling2D, concatenate, BatchNormalization, Lambda, LeakyReLU, TimeDistributed, Reshape, LSTM, GaussianNoise
from keras.models         import Model
from keras.datasets.mnist import load_data
from keras.utils          import to_categorical
from keras.initializers   import Orthogonal
from keras.regularizers   import L1L2, l2
from keras.callbacks      import Callback
import keras.backend as K
import tensorflow as tf

import loaders
from loaders.transform import AddGaussianNoise, Windowing, GSMize, MixReverb, MixNoise, ConstantQTransform, Transcript, LengthOfWindow, Length, Lengthen, Null, DivisiblePad
from loaders.dataset import Dataset, enable_multiprocessing, disable_multiprocessing
from loaders.feature import NormalizedLogPowerRFFT, Trim
from loaders.experiment import SingleExperiment
from loaders.measures import MSE
from loaders.schedule import Schedule
from loaders.estimator import DefaultVisualizeTransform, Estimator
from loaders.extras import LossHistory, set_device, delog_griffin_lim, get_stft
from loaders.custom import identity_loss
set_device()               # switches to CPU if GPU busy
enable_multiprocessing()   # for faster preprocessing of data (multicore)

Switching to CPU only as GPU is busy or unavailable


Process ForkPoolWorker-14:
Process ForkPoolWorker-11:
Process ForkPoolWorker-17:
KeyboardInterrupt
Process ForkPoolWorker-12:
Traceback (most recent call last):
Process ForkPoolWorker-13:
Traceback (most recent call last):
Process ForkPoolWorker-18:
Traceback (most recent call last):
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/multiprocess/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/site-packages/multiprocess/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/multiprocess/pool.py", line 108, in worker
  

##### Data loading

In [8]:
TRAIN = 9000
VALID = 80
TEST = 80

import numpy as np
import scipy.io.wavfile as sio
import gc

import tempfile
import subprocess

tempnam = tempfile.mktemp

log_fft_source = log_fft_target = phase = train = test = valid = experiment = None
gc.collect()

dataset = Dataset.from_folder("DAE-libri",
                              verbose = False,
                              dataset_pad = 8,
                              trim_lengths = 160000,
                              cache = "cache",
                              ram_cache_size = None,
                              filter=lambda x:x.endswith(".wav") and not x.endswith(".gsm.wav"))


log_fft_source = np.zeros([TEST, 1248, 129], np.float32)
log_fft_target = np.zeros([TEST, 1248, 257], np.float32)
phase = np.zeros([TEST, 1248, 129], np.float32)

for ix, file in enumerate(dataset.files):
    print(ix)
    data = sio.read(dataset.root + "/" + file)[1] # .astype(np.float32) to generate noise in the experiment...
    oldname = tempnam() + '.oldwav'
    sio.write(oldname, 16000, data)
    tmpname = tempnam() + '.amr-nb'
    newname = dataset.root + "/" + file + ".gsm.wav"
    subprocess.Popen(['sox', oldname, '-C', '0', '-r', '8000', tmpname]).communicate()
    subprocess.Popen(['sox', tmpname, '-r', '16000', "-e", "signed", '-b', '16',  newname]).communicate()
    print(tmpname)


0
/tmp/tmpxfj29kxv.amr-nb
1
/tmp/tmp4dz_x3sl.amr-nb
2
/tmp/tmpx5key7p8.amr-nb
3
/tmp/tmp9hdt6e_z.amr-nb
4
/tmp/tmplxksoq9l.amr-nb
5
/tmp/tmpcdurf4wg.amr-nb
6
/tmp/tmpuzn8zsle.amr-nb
7
/tmp/tmp35as6nig.amr-nb
8
/tmp/tmpe03r74kn.amr-nb
9
/tmp/tmpy0uexoaz.amr-nb
10
/tmp/tmpxretmik6.amr-nb
11
/tmp/tmpdvxifac_.amr-nb
12
/tmp/tmpv99jpone.amr-nb
13
/tmp/tmp2ou9l_12.amr-nb
14
/tmp/tmp00cxg5n8.amr-nb
15
/tmp/tmpmb5bg60k.amr-nb
16
/tmp/tmp7zpw5rko.amr-nb
17
/tmp/tmp06oyspgx.amr-nb
18


KeyboardInterrupt: 

In [3]:
MEAN = 0 # min([log_fft_source.min(), log_fft_target.min()])
# mean = log_fft_source.mean()
#log_fft_source -= MEAN
#log_fft_target -= MEAN
# STD = []
# for i in range(log_fft_source.shape[0]):
#     rec = (log_fft_source[i] - mean) ** 2
#     STD.append(rec.mean())
# STD = np.sqrt(np.array(STD).mean())
STD = log_fft_source.max() / 4.188281

log_fft_source /= STD
log_fft_target /= STD
print(log_fft_source.min(), log_fft_source.max())

0.0 4.188281


In [4]:
test = log_fft_source[:,:,:129], log_fft_target[:]

In [5]:
with open("/tmp/test_data.bin", "wb") as f:
    pickle.dump((test, phase), f)
    
with open("/tmp/data_norm.bin", "wb") as f:
    pickle.dump((MEAN, STD), f)

In [ ]:
# How to normalize

"""
Check how it differentiates with normalization
How does signal differentiates by normalization?
"""